In [ ]:
import requests

base_url = "https://api.waqi.info"
token = "eb5b6acae357a25be942b74897e4e18f3b371f24"

#on peux tester en mettant une ville 
city = 'Paris'
r = requests.get(base_url + f"/feed/{city}/?token={token}")
"City: {}, Air quality index: {}".format(r.json()['data']['city']['name'], r.json()['data']['aqi'])

#on peut faire chercher par cordonnées 
latlngbox = "41.3718,-5.1428,51.0870,9.5593" 
r = requests.get(base_url + f"/map/bounds/?latlng={latlngbox}&token={token}")
[f"Station: {d['station']['name']}, latitude: {d['lat']}, longitude: {d['lon']}, air quality: {d['aqi']}" for d in r.json()['data']]

#Transformer en Dataframe

import pandas as pd

def make_dataframe(r):
    """Extracts data from request r and returns a DataFrame."""
    rows = []
    for item in r.json()['data']:
        rows.append([item['lat'], item['lon'], item['aqi'], item['station']['name']])
    df = pd.DataFrame(rows, columns=['lat', 'lon', 'aqi', 'name'])
    df['aqi'] = pd.to_numeric(df.aqi, errors='coerce')
    return df

df = make_dataframe(r)
df.head()

#on fait  un scatter plot avec holoviews 
import holoviews as hv
hv.extension('bokeh', logo=False)

scatter = hv.Scatter(df.dropna(), kdims='lon', vdims=['lat', 'aqi', 'name'])
scatter.opts(color='aqi', size=10, padding=.1, tools=['hover'], colorbar=True, cmap='magma', width=500, height=400, clim=(0, 60))